In [1]:
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter, TokenTextSplitter
from ragas.metrics import faithfulness, answer_relevancy, context_precision, context_recall
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain_experimental.text_splitter import SemanticChunker
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_text_splitters import MarkdownHeaderTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_core.documents import Document
from langchain_chroma import Chroma
from dotenv import load_dotenv
from datasets import Dataset
from ragas import evaluate
from pathlib import Path
import numpy as np
import shutil
import os


# Cargar variables desde el archivo .env
load_dotenv()

# Asignar la clave de API a la variable de entorno que espera el SDK de Google
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")

**Table of contents**<a id='toc0_'></a>    
- [Ingestion de documentos](#toc1_)    
  - [Carga de documentos en formato MarkDown](#toc1_1_)    
  - [Segmentación (chunking) de la información](#toc1_2_)    
    - [Segmentación basada en longitud](#toc1_2_1_)    
    - [Segmentación basada en estructura de texto](#toc1_2_2_)    
    - [Segmentación basada en estructura de documento](#toc1_2_3_)    
    - [Segmentación basada en significado semántico](#toc1_2_4_)    
- [Carga de chunks a base de datos vectorial](#toc2_)    
  - [Creación de chunks finales para todos los documentos](#toc2_1_)    
  - [Creación de colección y embeddings](#toc2_2_)    
    - [Test del Vectorstore](#toc2_2_1_)    
- [Construcción del retriever](#toc3_)    
  - [Test del Retriever](#toc3_1_)    
  - [Generación de respuestas con RAG](#toc3_2_)    
    - [Creación del Modelo](#toc3_2_1_)    
    - [Pregunta al RAG](#toc3_2_2_)    
    - [Mostrar resultados del RAG](#toc3_2_3_)    
    - [Prueba del RAG](#toc3_2_4_)    
  - [Evaluación del sistema RAG](#toc3_3_)    
    - [ Crea dataset con ground truth basado ÚNICAMENTE en información que realmente existe en los documentos proporcionados](#toc3_3_1_)    
    - [Crea un dataset de evaluación solo con preguntas cuya respuesta puede verificarse con los documentos disponibles (ground truth verificado).](#toc3_3_2_)    
    - [Evalúa el sistema RAG utilizando las métricas definidas por RAGAS](#toc3_3_3_)    
    - [Resumen de métricas RAGAS promediada](#toc3_3_4_)    
    - [Corre una evalución del RAGAS](#toc3_3_5_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[Ingestion de documentos](#toc0_)


## <a id='toc1_1_'></a>[Carga de documentos en formato MarkDown](#toc0_)


In [2]:
# Ruta al archivo Markdown de ejemplo que se va a cargar
EXAMPLE_MD_PATH = "data/development_policies.md"


def load_single_markdown(file_path, mode):
    """
Carga un archivo Markdown individual y lo convierte en una lista de objetos Document.

Parámetros:
- file_path: ruta al archivo Markdown.
- mode: modo de carga ('single' para todo el contenido como un solo documento,
        'elements' para dividirlo en fragmentos).

Retorna:
- Lista de objetos Document.
"""

    # Inicializa el loader con el archivo y el modo especificado
    loader = UnstructuredMarkdownLoader(file_path, mode=mode)

    # Carga el contenido del archivo como documentos
    data = loader.load()

    # Muestra información sobre el resultado de la carga
    print(f"\nArchivo cargado: {file_path}")
    print(f"Número de documentos generados: {len(data)}")
    print(f"Tipo del primer elemento: {type(data[0])}")

    return data


# Carga el archivo como un único documento (modo 'single')
data = load_single_markdown(
    EXAMPLE_MD_PATH,
    mode="single"
)

# Muestra los primeros 500 caracteres del contenido del documento cargado
print(f"Contenido del documento:\n\n\n{data[0].page_content[:500]}...")


Archivo cargado: data/development_policies.md
Número de documentos generados: 1
Tipo del primer elemento: <class 'langchain_core.documents.base.Document'>
Contenido del documento:


Políticas y Procedimientos de Desarrollo

1. POLÍTICA DE CONTROL DE VERSIONES

1.1 Propósito

Establecer directrices claras para el manejo del código fuente y garantizar la trazabilidad de cambios en todos los proyectos de desarrollo.

1.2 Alcance

Esta política aplica a todo el personal de desarrollo, QA y DevOps involucrado en proyectos de software.

1.3 Herramientas Aprobadas

Sistema de Control de Versiones: Git

Repositorios Centralizados: GitHub Enterprise / GitLab

Herramientas de Revisió...


In [3]:
# Carga el archivo Markdown en modo 'elements', lo que divide el contenido
# en múltiples objetos Document según su estructura (títulos, listas, párrafos, etc.)
elements = load_single_markdown(
    EXAMPLE_MD_PATH,
    mode="elements"
)

print("\n" + "--" * 40 + "\n")

# Itera sobre los primeros 10 fragmentos generados
for i, element in enumerate(elements[:10]):
    print(f"\n** Documento {i+1} **\n")

    # Imprime la categoría del fragmento si está disponible (proporcionada por el loader)
    print(f"Categoría: {element.metadata.get('category', 'N/A')}")

    # Imprime el contenido textual del fragmento
    print(f"Contenido: {element.page_content}\n")
    print("--" * 40 + "\n")


Archivo cargado: data/development_policies.md
Número de documentos generados: 81
Tipo del primer elemento: <class 'langchain_core.documents.base.Document'>

--------------------------------------------------------------------------------


** Documento 1 **

Categoría: Title
Contenido: Políticas y Procedimientos de Desarrollo

--------------------------------------------------------------------------------


** Documento 2 **

Categoría: Title
Contenido: 1. POLÍTICA DE CONTROL DE VERSIONES

--------------------------------------------------------------------------------


** Documento 3 **

Categoría: Title
Contenido: 1.1 Propósito

--------------------------------------------------------------------------------


** Documento 4 **

Categoría: NarrativeText
Contenido: Establecer directrices claras para el manejo del código fuente y garantizar la trazabilidad de cambios en todos los proyectos de desarrollo.

------------------------------------------------------------------------------

In [4]:
def load_markdown_documents(data_folder="data"):
    """
Carga todos los archivos .md de una carpeta específica.

Args:
    data_folder (str): Ruta a la carpeta que contiene los archivos Markdown

Returns:
    list: Lista de objetos Document, uno por cada archivo cargado
"""

    documents = []
    data_path = Path(data_folder)

    # Verifica que la carpeta exista
    if not data_path.exists():
        raise FileNotFoundError(f"\nLa carpeta '{data_folder}' no existe")

    # Busca todos los archivos con extensión .md en la carpeta
    markdown_files = list(data_path.glob("*.md"))

    # Lanza un error si no se encuentran archivos Markdown
    if not markdown_files:
        raise FileNotFoundError(
            f"\nNo se encontraron archivos .md en '{data_folder}'")

    print(f"\nArchivos Markdown encontrados: {len(markdown_files)}")
    print("=" * 50)

    # Itera sobre los archivos encontrados, ordenados alfabéticamente
    for file_path in sorted(markdown_files):
        try:
            # Inicializa el loader para el archivo actual
            loader = UnstructuredMarkdownLoader(str(file_path))

            # Carga el contenido como lista de Document (modo por defecto: 'single')
            data = loader.load()

            # Toma el primer documento generado (asumiendo modo 'single')
            document = data[0]

            # Agrega metadatos adicionales al documento
            document.metadata.update({
                "source_file": file_path.name,
                "file_size": file_path.stat().st_size,
                "total_characters": len(document.page_content)
            })

            # Añade el documento a la lista final
            documents.append(document)

            # Imprime detalles del archivo cargado
            print(f"\n✓ {file_path.name}")
            print(f"  Caracteres: {len(document.page_content):,}")
            print(f"  Tamaño archivo: {file_path.stat().st_size:,} bytes")

        except Exception as e:
            # Captura e informa cualquier error durante la carga de archivos
            print(f"\n✗ Error cargando {file_path.name}: {str(e)}")
            continue

    print("\n" + "=" * 50)
    print(f"Total documentos cargados exitosamente: {len(documents)}")

    return documents

    # Carga todos los archivos Markdown en la carpeta "data"


docs = load_markdown_documents("data")


Archivos Markdown encontrados: 5

✓ api_development_standards.md
  Caracteres: 4,908
  Tamaño archivo: 5,522 bytes

✓ development_policies.md
  Caracteres: 6,646
  Tamaño archivo: 7,144 bytes

✓ multi_theme_descriptions.md
  Caracteres: 6,324
  Tamaño archivo: 6,458 bytes

✓ software_architecture_guide.md
  Caracteres: 2,694
  Tamaño archivo: 2,911 bytes

✓ troubleshooting_guide.md
  Caracteres: 10,960
  Tamaño archivo: 11,548 bytes

Total documentos cargados exitosamente: 5


## <a id='toc1_2_'></a>[Segmentación (chunking) de la información](#toc0_)

### <a id='toc1_2_1_'></a>[Segmentación basada en longitud](#toc0_)

In [5]:
def demonstrate_length_based_chunking(documents):
    
        """
        Demuestra dos estrategias de chunking basadas en longitud: por caracteres y por tokens.

        Args:
                documents (list): Lista de objetos Document a procesar.
                
        Returns:
                tuple: Dos listas de chunks (por caracteres y por tokens).
        """
                
        print("\n" + "=" * 70)
        print("1. SEGMENTACIÓN BASADA EN LONGITUD")
        print("=" * 70)

        # Se toma el primer documento de ejemplo para realizar la demostración
        sample_doc = documents[0]

        # 1.1 SEGMENTACION BASADO EN CARACTERES
        print("\n1.1 SEGMENTACIÓN BASADA EN CARACTERES")
        print("-" * 40)

        char_splitter = CharacterTextSplitter(
                chunk_size = 500,       # Máximo de 500 caracteres por fragmento
                chunk_overlap = 100,    # 100 caracteres de solapamiento entre fragmentos
                separator = "\n\n"      # Se intenta separar por párrafos si es posible
        )

        # Se genera la lista de fragmentos a partir del documento
        char_chunks = char_splitter.split_documents([sample_doc])

        # Estadísticas sobre el resultado
        print(f"Documento original: {len(sample_doc.page_content):,} caracteres")
        print(f"Chunks generados: {len(char_chunks)}")
        print(f"Tamaño promedio: {np.mean([len(chunk.page_content) for chunk in char_chunks]):.0f} caracteres")

        # Muestra los dos primeros fragmentos generados
        for i, chunk in enumerate(char_chunks[:2]):
                print("\n" + "-" * 80) 
                print(f"\nChunk {i+1} ({len(chunk.page_content)} caracteres):\n")
                print(chunk.page_content.strip())  

        print("\n" + "-" * 80) 

        # 1.2 SEGMENTACION BASADA EN TOKENS
        print("\n1.2 SEGMENTACION BASADA EN TOKENS")
        print("-" * 40)

        token_splitter = TokenTextSplitter(
                chunk_size = 150,       # Máximo de 150 tokens por fragmento
                chunk_overlap = 20      # 20 tokens de solapamiento entre fragmentos
        )

        # Se genera la lista de fragmentos usando tokenización
        token_chunks = token_splitter.split_documents([sample_doc])

        # Estadísticas sobre el resultado
        print(f"Documento original: {len(sample_doc.page_content):,} caracteres")
        print(f"Chunks generados: {len(token_chunks)}")
        print(f"Tamaño promedio: {np.mean([len(chunk.page_content) for chunk in token_chunks]):.0f} caracteres")

        # Muestra los dos primeros fragmentos generados
        for i, chunk in enumerate(char_chunks[:2]):
                print("\n" + "-" * 80) 
                print(f"\nChunk {i+1} ({len(chunk.page_content)} caracteres):\n")
                print(chunk.page_content.strip())  

        print("\n" + "-" * 80 + "\n") 

        return char_chunks, token_chunks

# Ejecuta la función usando la lista de documentos previamente cargados
demonstrate_length_based_chunking(docs)
print()


1. SEGMENTACIÓN BASADA EN LONGITUD

1.1 SEGMENTACIÓN BASADA EN CARACTERES
----------------------------------------
Documento original: 4,908 caracteres
Chunks generados: 13
Tamaño promedio: 441 caracteres

--------------------------------------------------------------------------------

Chunk 1 (448 caracteres):

Estándares de Desarrollo de APIs

Diseño RESTful

Principios REST

REST (Representational State Transfer) es un estilo arquitectónico para servicios web que define las siguientes restricciones:

Cliente-Servidor: Separación clara de responsabilidades

Sin Estado: Cada solicitud debe contener toda la información necesaria

Cacheable: Las respuestas deben ser marcadas como cacheables o no

Interfaz Uniforme: Uso consistente de métodos HTTP y URIs

--------------------------------------------------------------------------------

Chunk 2 (449 caracteres):

Interfaz Uniforme: Uso consistente de métodos HTTP y URIs

Sistema en Capas: La arquitectura puede estar compuesta por múltip

### <a id='toc1_2_2_'></a>[Segmentación basada en estructura de texto](#toc0_)

In [6]:
def demonstrate_text_structured_chunking(documents):
    
        """
    Demuestra la estrategia de segmentación basada en la estructura del texto
    utilizando el RecursiveCharacterTextSplitter de LangChain.

    Args:
        documents (list): Lista de objetos Document a procesar.

    Returns:
        list: Lista de fragmentos generados a partir del primer documento.
    """

        print("\n" + "=" * 70)
        print("2. SEGMENTACIÓN BASADA EN ESTRUCTURA DEL TEXTO")
        print("=" * 70)

        # Se toma el primer documento como muestra para la demostración
        sample_doc = documents[0]

        # Se configura un splitter recursivo que prioriza separaciones semánticas
        recursive_splitter = RecursiveCharacterTextSplitter(
                chunk_size = 1000,       # Tamaño máximo por fragmento
                chunk_overlap = 500,     # Solapamiento entre fragmentos
                separators = [           # Separadores en orden de prioridad
                        "\n\n",    # Párrafos
                        "\n",      # Líneas
                        ". ",      # Oraciones
                        ", ",      # Cláusulas
                        " ",       # Palabras
                        ""         # Caracteres individuales
                ]
        )

        # Se generan los fragmentos aplicando la estrategia recursiva
        recursive_chunks = recursive_splitter.split_documents([sample_doc])

        print(f"\nDocumento original: {len(sample_doc.page_content):,} caracteres")
        print(f"Chunks generados: {len(recursive_chunks)}")

        # Se calcula el tamaño promedio de los fragmentos resultantes
        chunk_sizes = [len(chunk.page_content) for chunk in recursive_chunks]
        print(f"Tamaño promedio: {np.mean(chunk_sizes):.0f} caracteres")

        # Se imprimen los primeros 3 fragmentos generados, con contenido visible
        for i, chunk in enumerate(recursive_chunks[:3]):
                print("\n" + "-" * 80) 
                print(f"\nChunk {i+1} ({len(chunk.page_content)} chars):")
                print(f"'{chunk.page_content}'")
        
        print("\n" + "-" * 80 + "\n") 

        return recursive_chunks

# Se ejecuta la demostración con la lista de documentos cargados
demonstrate_text_structured_chunking(docs)
print()


2. SEGMENTACIÓN BASADA EN ESTRUCTURA DEL TEXTO

Documento original: 4,908 caracteres
Chunks generados: 9
Tamaño promedio: 902 caracteres

--------------------------------------------------------------------------------

Chunk 1 (840 chars):
'Estándares de Desarrollo de APIs

Diseño RESTful

Principios REST

REST (Representational State Transfer) es un estilo arquitectónico para servicios web que define las siguientes restricciones:

Cliente-Servidor: Separación clara de responsabilidades

Sin Estado: Cada solicitud debe contener toda la información necesaria

Cacheable: Las respuestas deben ser marcadas como cacheables o no

Interfaz Uniforme: Uso consistente de métodos HTTP y URIs

Sistema en Capas: La arquitectura puede estar compuesta por múltiples capas

Métodos HTTP y su Uso

Método Propósito Idempotente Seguro GET Obtener recursos Sí Sí POST Crear recursos No No PUT Actualizar/crear recursos Sí No PATCH Actualización parcial No No DELETE Eliminar recursos Sí No

Convenciones de 

### <a id='toc1_2_3_'></a>[Segmentación basada en estructura de documento](#toc0_)

In [7]:
def load_markdown_for_structure_splitting(file_path):
    
        """
    Carga el contenido completo de un archivo Markdown como texto plano.
    
    Args:
        file_path (str): Ruta del archivo Markdown a cargar.
        
    Returns:
        Document: Objeto Document con el contenido y metadatos básicos.
    """
    
    # Abrir el archivo en modo lectura y codificación UTF-8
        with open(file_path, 'r', encoding='utf-8') as file:
                content = file.read()

        # Retornar el contenido como objeto Document sin procesamiento estructural
        return Document(
                page_content = content,
                metadata={"source": file_path}
        )


def demonstrate_document_structured_chunking(file_path):

        """
        Aplica segmentación basada en la estructura del documento, utilizando encabezados Markdown.

        Args:
                file_path (str): Ruta del archivo Markdown a procesar.
                
        Returns:
                list: Lista de fragmentos generados a partir de los encabezados Markdown.
        """
    
        # Cargar el archivo sin procesamiento previo para preservar su estructura original
        raw_doc = load_markdown_for_structure_splitting(file_path)

        print("\n" + "=" * 70)
        print("3. SEGMENTACIÓN BASADA EN ESTRUCTURA DEL DOCUMENTO")
        print("=" * 70)

        # Definir el segmentador que dividirá el texto por encabezados Markdown
        markdown_splitter = MarkdownHeaderTextSplitter(
                headers_to_split_on=[("#", "Header 1"), ("##", "Header 2"), ("###", "Header 3")],
                strip_headers=False  # Se conservan los encabezados dentro de los fragmentos
        )

        # Aplicar el segmentador al contenido del documento
        md_chunks = markdown_splitter.split_text(raw_doc.page_content)

        print(f"\nDocumento original: {len(raw_doc.page_content):,} caracteres")
        # Imprimir la cantidad de fragmentos generados (debe ser mayor a 1 si hay encabezados)
        print(f"Chunks generados: {len(md_chunks)}")

        return md_chunks


# Ejecutar la segmentación sobre un archivo de ejemplo
chunks = demonstrate_document_structured_chunking("data/api_development_standards.md")

# Mostrar los primeros 5 fragmentos resultantes
for i, chunk in enumerate(chunks[:5]):
        print("\n" + "-" * 80) 
        print(f"\nChunk {i+1} ({len(chunk.page_content)} chars):")
        print(f"'{chunk.page_content}'")

print("\n" + "-" * 80 + "\n") 


3. SEGMENTACIÓN BASADA EN ESTRUCTURA DEL DOCUMENTO

Documento original: 5,462 caracteres
Chunks generados: 20

--------------------------------------------------------------------------------

Chunk 1 (567 chars):
'# Estándares de Desarrollo de APIs  
## Diseño RESTful  
### Principios REST
REST (Representational State Transfer) es un estilo arquitectónico para servicios web que define las siguientes restricciones:  
1. **Cliente-Servidor**: Separación clara de responsabilidades
2. **Sin Estado**: Cada solicitud debe contener toda la información necesaria
3. **Cacheable**: Las respuestas deben ser marcadas como cacheables o no
4. **Interfaz Uniforme**: Uso consistente de métodos HTTP y URIs
5. **Sistema en Capas**: La arquitectura puede estar compuesta por múltiples capas'

--------------------------------------------------------------------------------

Chunk 2 (323 chars):
'### Métodos HTTP y su Uso  
| Método | Propósito | Idempotente | Seguro |
|--------|-----------|-------------|

### <a id='toc1_2_4_'></a>[Segmentación basada en significado semántico](#toc0_)

In [8]:
def demonstrate_semantic_chunking(documents):
    
        """
    Demuestra segmentación semántica (semantic chunking) usando embeddings
    y una estrategia basada en percentiles para detectar transiciones de significado.

    Args:
        documents (list): Lista de objetos Document a procesar.

    Returns:
        dict: Diccionario con las listas de chunks generadas para cada configuración.
    """
    
        print("\n" + "=" * 70)
        print("4. SEGMENTACIÓN BASADA EN SIGNIFICADO SEMÁNTICO")
        print("=" * 70)

        # Cargar el modelo de embeddings de Google
        embeddings_model = GoogleGenerativeAIEmbeddings(
                model = "models/embedding-001"
        )

        # Seleccionar un documento de ejemplo
        sample_doc = documents[2]
        print(f"\nAnalizando: {sample_doc.metadata.get('source_file', 'documento')}")
        print(f"Tamaño original: {len(sample_doc.page_content):,} caracteres")

        # Definir configuraciones con diferentes umbrales de percentile
        configurations = {
                "percentile_95": {
                        "chunker": SemanticChunker(
                                embeddings_model, 
                                breakpoint_threshold_type = "percentile",
                                breakpoint_threshold_amount = 95.0
                        ),
                        "description": "Conservador (percentil 95)"
                },
                "percentile_85": {
                        "chunker": SemanticChunker(
                                embeddings_model, 
                                breakpoint_threshold_type = "percentile",
                                breakpoint_threshold_amount = 85.0
                        ),
                        "description": "Agresivo (percentil 85)"
                }
        }

        results = {}

        print("\nComparando configuraciones de percentil:")
        print("-" * 80)

        # Ejecutar segmentación con cada configuración
        for config_name, config_info in configurations.items():
                print(f"\nConfiguración: {config_info['description']}")
                
                try:
                        # Aplicar segmentación semántica sobre el contenido textual
                        chunks = config_info['chunker'].create_documents([sample_doc.page_content])
                        
                        # Enriquecer con metadatos
                        for i, chunk in enumerate(chunks):
                                chunk.metadata.update({
                                        "chunk_type": f"semantic_{config_name}",
                                        "chunk_index": i,
                                        "threshold": config_info['description'],
                                        "source_document": sample_doc.metadata.get("source_file", "unknown")
                                })
                        
                        # Estadísticas sobre los chunks generados
                        chunk_sizes = [len(chunk.page_content) for chunk in chunks]
                        print(f"  \nChunks generados: {len(chunks)}")
                        print(f"  Tamaño promedio: {np.mean(chunk_sizes):.0f} caracteres")
                        print(f"  Rango: {min(chunk_sizes)} - {max(chunk_sizes)} caracteres")
                        
                        # Mostrar vista previa de los primeros chunks
                        print(f"  \nEjemplos de chunks:")
                        for i, chunk in enumerate(chunks[:5]):
                                preview = chunk.page_content[:100].replace('\n', ' ')
                                print(f"    Chunk {i+1}: '{preview}...'")
                        
                        results[config_name] = chunks

                        print("\n" + "-" * 80)

                except Exception as e:
                        print(f"  Error al procesar: {str(e)}")
                        results[config_name] = []

        return results


# Ejecutar la función usando los documentos cargados previamente
semantic_chunks = demonstrate_semantic_chunking(docs)


4. SEGMENTACIÓN BASADA EN SIGNIFICADO SEMÁNTICO

Analizando: multi_theme_descriptions.md
Tamaño original: 6,324 caracteres

Comparando configuraciones de percentil:
--------------------------------------------------------------------------------

Configuración: Conservador (percentil 95)


E0000 00:00:1760853377.232042  203985 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


  
Chunks generados: 4
  Tamaño promedio: 1577 caracteres
  Rango: 236 - 3920 caracteres
  
Ejemplos de chunks:
    Chunk 1: 'Manual Técnico de Desarrollo de Software  Desarrollo de APIs REST  Las APIs REST representan el está...'
    Chunk 2: 'Los códigos de respuesta HTTP proporcionan información clara sobre el resultado de cada operación, d...'
    Chunk 3: 'Los tokens JWT contienen información del usuario codificada y firmada, permitiendo verificación sin ...'
    Chunk 4: 'El principio de menor privilegio limita accesos a lo mínimo necesario para cada rol. El cifrado en t...'

--------------------------------------------------------------------------------

Configuración: Agresivo (percentil 85)
  
Chunks generados: 10
  Tamaño promedio: 630 caracteres
  Rango: 95 - 1823 caracteres
  
Ejemplos de chunks:
    Chunk 1: 'Manual Técnico de Desarrollo de Software  Desarrollo de APIs REST  Las APIs REST representan el está...'
    Chunk 2: 'Los códigos de respuesta HTTP proporcionan inf

# <a id='toc2_'></a>[Carga de chunks a base de datos vectorial](#toc0_)

## <a id='toc2_1_'></a>[Creación de chunks finales para todos los documentos](#toc0_)

In [9]:
def load_and_chunk_markdown_documents(data_folder = "data"):
        """
    Carga todos los documentos Markdown desde la carpeta especificada,
    aplicando segmentación basada en estructura de encabezados para 
    generar chunks semánticamente coherentes, listos para el proceso 
    de embedding.

    Args:
        data_folder (str): Carpeta que contiene los archivos .md

    Returns:
        list: Lista de objetos Document (chunks) enriquecidos con metadatos
    """
    
        print("\nCARGA Y CHUNKING DE DOCUMENTOS EN FORMATO MARKDOWN")
        print("=" * 70)

        # Definir niveles jerárquicos que guiarán la segmentación
        headers_to_split_on = [
                ("#", "Header 1"),
                ("##", "Header 2"), 
                ("###", "Header 3"),
                ("####", "Header 4"),
        ]

        markdown_splitter = MarkdownHeaderTextSplitter(
                headers_to_split_on = headers_to_split_on,
                strip_headers = False
        )

        # Verificar existencia de la carpeta y archivos Markdown
        data_path = Path(data_folder)
        if not data_path.exists():
                raise FileNotFoundError(f"\nLa carpeta '{data_folder}' no existe")

        markdown_files = list(data_path.glob("*.md"))
        if not markdown_files:
                raise FileNotFoundError(f"\nNo se encontraron archivos .md en '{data_folder}'")

        all_chunks = []

        print(f"\nProcesando {len(markdown_files)} archivos:\n")
        print("-" * 30)

        for file_path in sorted(markdown_files):
                try:
                        # Leer contenido crudo del archivo
                        with open(file_path, 'r', encoding='utf-8') as file:
                                content = file.read()
                        
                        # Aplicar segmentación estructural
                        chunks = markdown_splitter.split_text(content)
                        
                        # Enriquecer cada chunk con metadatos útiles
                        for i, chunk in enumerate(chunks):
                                chunk.metadata.update({
                                        "source_file": file_path.name,
                                        "source_path": str(file_path),
                                        "chunk_index": i,
                                        "total_chunks_in_doc": len(chunks),
                                        "chunking_strategy": "document_structured",
                                        "file_size": file_path.stat().st_size,
                                        "chunk_size": len(chunk.page_content)
                                })
                        
                        all_chunks.extend(chunks)
                        print(f"{file_path.name}: {len(chunks)} chunks generados")

                except Exception as e:
                        print(f"\nError procesando {file_path.name}: {str(e)}")
                        continue

        print("-" * 30)
        print(f"Total chunks generados: {len(all_chunks)}")

        # Mostrar estadísticas generales
        if all_chunks:
                chunk_sizes = [len(chunk.page_content) for chunk in all_chunks]
                print(f"Tamaño promedio de chunks: {sum(chunk_sizes) / len(chunk_sizes):.0f} caracteres")
                print(f"Rango de tamaños: {min(chunk_sizes)} - {max(chunk_sizes)} caracteres")
                print("-" * 30)

        return all_chunks


def preview_chunks(chunks, num_examples = 5):
    
        """
    Muestra una vista previa de los primeros chunks generados.

    Args:
        chunks (list): Lista de objetos Document generados
        num_examples (int): Número de ejemplos a mostrar
    """
    
        print(f"\nVISTA PREVIA DE CHUNKS ({num_examples} ejemplos):")
        print("=" * 70)

        for i, chunk in enumerate(chunks[:num_examples]):
                print(f"\nChunk {i+1}:")
                print(f"Archivo: {chunk.metadata.get('source_file', 'unknown')}")
                print(f"Headers: {[f'{k}: {v}' for k, v in chunk.metadata.items() if 'Header' in k]}")
                print(f"Tamaño: {len(chunk.page_content)} caracteres")
                
                # Mostrar primeras líneas del contenido
                content_preview = chunk.page_content[:200].replace('\n', ' ')
                print(f"Contenido: '{content_preview}...'")
                print("-" * 40)

In [10]:
def prepare_documents_for_rag(data_folder = "data"):
    """
    Función completa que prepara documentos para el pipeline RAG.
    
    Returns:
        list: Chunks listos para crear embeddings y guardar en base de datos vectorial
    """
    
    # Cargar y hacer chunking de todos los documentos
    chunks = load_and_chunk_markdown_documents(data_folder)
    
    # Mostrar algunos ejemplos para validación rápida
    preview_chunks(chunks)
    
    return chunks


final_chunks = prepare_documents_for_rag()


CARGA Y CHUNKING DE DOCUMENTOS EN FORMATO MARKDOWN

Procesando 5 archivos:

------------------------------
api_development_standards.md: 20 chunks generados
development_policies.md: 28 chunks generados
multi_theme_descriptions.md: 7 chunks generados
software_architecture_guide.md: 14 chunks generados
troubleshooting_guide.md: 8 chunks generados
------------------------------
Total chunks generados: 77
Tamaño promedio de chunks: 413 caracteres
Rango de tamaños: 94 - 1692 caracteres
------------------------------

VISTA PREVIA DE CHUNKS (5 ejemplos):

Chunk 1:
Archivo: api_development_standards.md
Headers: ['Header 1: Estándares de Desarrollo de APIs', 'Header 2: Diseño RESTful', 'Header 3: Principios REST']
Tamaño: 567 caracteres
Contenido: '# Estándares de Desarrollo de APIs   ## Diseño RESTful   ### Principios REST REST (Representational State Transfer) es un estilo arquitectónico para servicios web que define las siguientes restriccion...'
----------------------------------------

C

## <a id='toc2_2_'></a>[Creación de colección y embeddings](#toc0_)

In [11]:
def load_to_chromadb(chunks, collection_name = "rag_docs", persist_directory = "./chroma_db"):
     
        """
    Carga una lista de chunks en una base de datos vectorial ChromaDB utilizando LangChain.
    
    Esta función prepara la colección eliminando cualquier contenido previo,
    genera embeddings para cada fragmento textual y los guarda junto con metadatos e identificadores únicos.
    
    Args:
        chunks (list): Lista de objetos Document generados previamente por el proceso de chunking.
        collection_name (str): Nombre lógico para la colección en la base de datos vectorial.
        persist_directory (str): Ruta local donde se almacenará la colección de forma persistente.

    Returns:
        Chroma: Objeto de almacenamiento vectorial que permite realizar consultas semánticas.
    """
    
        # Eliminar la base anterior si existe para asegurar una carga limpia
        if os.path.exists(persist_directory):
                shutil.rmtree(persist_directory)

        # Instanciar el modelo de embeddings de Google
        embeddings = GoogleGenerativeAIEmbeddings(model = "models/embedding-001")

        # Crear una base de datos vectorial persistente con la colección especificada
        vector_store = Chroma(
                collection_name = collection_name,
                embedding_function = embeddings,
                persist_directory = persist_directory
        )

        # Extraer los textos de cada chunk
        texts = [chunk.page_content for chunk in chunks]

        # Extraer los metadatos asociados (por ejemplo: archivo fuente, tamaño, headers)
        metadatas = [chunk.metadata for chunk in chunks]

        # Generar identificadores únicos para cada chunk
        ids = [f"{chunk.metadata.get('source_file', 'doc')}_{i}" for i, chunk in enumerate(chunks)]

        # Agregar los embeddings y metadatos a la colección
        vector_store.add_texts(texts=texts, metadatas=metadatas, ids=ids)

        print(f"\nChromaDB: {len(chunks)} chunks cargados")
        return vector_store

In [12]:
vector_store = load_to_chromadb(final_chunks)

E0000 00:00:1760853379.747346  203985 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.



ChromaDB: 77 chunks cargados


### <a id='toc2_2_1_'></a>[Test del Vectorstore](#toc0_)

In [13]:
def test_vectorstore(vector_store, queries = None):
    
        """
    Realiza consultas de prueba sobre una colección vectorial para verificar su funcionalidad.
    
    Ejecuta una búsqueda semántica por similitud utilizando preguntas comunes y muestra
    el documento más relevante encontrado en la base de datos.

    Args:
        vector_store (Chroma): Objeto de almacenamiento vectorial cargado previamente.
        queries (list): Lista opcional de preguntas para probar la recuperación semántica.
    """
    
        if queries is None:
                queries = ["¿Qué son las APIs REST?", "¿Cómo funciona JWT?"]

        print("\nPROBANDO VECTOR STORE:")
        print("=" * 70)

        for query in queries:
                        # Obtener el chunk más relevante
                results = vector_store.similarity_search(query, k = 1) 
                if results:
                        doc = results[0]
                        file = doc.metadata.get('source_file', 'unknown')
                        preview = doc.page_content[:150]
                        print("\n" + "-" * 30)
                        print(f"\n  '{query}' → {file}: '{preview}...'")

        print("\n" + "-" * 30)

In [14]:
test_vectorstore(vector_store)


PROBANDO VECTOR STORE:

------------------------------

  '¿Qué son las APIs REST?' → api_development_standards.md: '# Estándares de Desarrollo de APIs  
## Diseño RESTful  
### Principios REST
REST (Representational State Transfer) es un estilo arquitectónico para s...'

------------------------------

  '¿Cómo funciona JWT?' → api_development_standards.md: '## Autenticación y Autorización  
### JWT (JSON Web Tokens)
Estructura de un JWT:
```
Header.Payload.Signature
```  
**Ejemplo de Header:**
```json
{
...'

------------------------------


# <a id='toc3_'></a>[Construcción del retriever](#toc0_)

In [15]:
def create_retriever(vector_store, k = 3, search_type = "similarity"):
    
        """
    Crea un objeto Retriever a partir del vector store para el sistema RAG.
    
    Args:
        vector_store: Instancia de Chroma o cualquier base vectorial compatible.
        k (int): Número de documentos a recuperar por consulta.
        search_type (str): Tipo de búsqueda a realizar (por similitud, MMR, etc.)
    
    Returns:
        retriever: Objeto LangChain retriever listo para usar.
    """
    
        # Generar el retriever a partir del vector store con los parámetros indicados
        retriever = vector_store.as_retriever(
                search_type = search_type,
                search_kwargs = {"k": k}  # Número de resultados por consulta
        )

        # Mostrar configuración para verificación rápida
        print(f"Retriever: {search_type}, k = {k}")

        return retriever

In [16]:
 # Crear retriever con búsqueda por similitud y k=3 resultados
retriever = create_retriever(vector_store, k=3)

Retriever: similarity, k = 3


## <a id='toc3_1_'></a>[Test del Retriever](#toc0_)

In [17]:
def test_retriever(retriever, query = "¿Cómo implementar autenticación JWT?"):
    
        """
    Prueba el retriever con una consulta.
    
    Args:
        retriever: Objeto retriever configurado previamente.
        query (str): Consulta de ejemplo para recuperar documentos.
    
    Returns:
        list: Lista de documentos recuperados.
    """
    
        # Ejecuta la consulta y recupera documentos relevantes
        docs = retriever.invoke(query)

        print("\nPROBANDO RETRIEVER:")
        print("=" * 70)

        # Muestra la consulta y la cantidad de documentos recuperados
        print(f"\nConsulta: '{query}'")
        print(f"Documentos recuperados: {len(docs)}")

        # Imprime una vista previa de cada documento recuperado
        for i, doc in enumerate(docs, 1):
                file = doc.metadata.get('source_file', 'unknown')  # Nombre del archivo original
                preview = doc.page_content[:60]  # Fragmento del contenido
                print("\n" + "-" * 30)
                print(f"\n  {i}. {file}: '{preview}...'")

        print("\n" + "-" * 30)

        return docs  # Devuelve la lista de documentos recuperados

In [18]:
test_retriever(retriever)
print()


PROBANDO RETRIEVER:

Consulta: '¿Cómo implementar autenticación JWT?'
Documentos recuperados: 3

------------------------------

  1. api_development_standards.md: '## Autenticación y Autorización  
### JWT (JSON Web Tokens)
...'

------------------------------

  2. api_development_standards.md: '### Implementación de Autenticación
1. **Bearer Token**: `Au...'

------------------------------

  3. software_architecture_guide.md: '### Implementación
- Autenticación y autorización
- Cifrado ...'

------------------------------



## <a id='toc3_2_'></a>[Generación de respuestas con RAG](#toc0_)

### <a id='toc3_2_1_'></a>[Creación del Modelo](#toc0_)

In [19]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

# Crear prompt personalizado en español.
# El LLM debe responder únicamente usando la información contextual recuperada.
prompt = ChatPromptTemplate.from_template("""
Eres un asistente especializado en responder preguntas sobre documentación técnica de desarrollo de software. Utiliza únicamente la información del contexto proporcionado para 
responder la pregunta. Si no conoces la respuesta basándote en el contexto, indica claramente que no tienes esa información. Mantén las respuestas concisas, precisas y usa máximo 
tres oraciones.

Pregunta: {question}

Contexto: {context}

Respuesta:
""")

# Configurar modelo de lenguaje de Google (gemini-2.5-flash)
# con baja temperatura para respuestas más deterministas.
llm = ChatGoogleGenerativeAI(
    model = "gemini-2.5-flash",
    temperature = 0.1,
    max_tokens = 200
)

E0000 00:00:1760853382.913024  203985 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


### <a id='toc3_2_2_'></a>[Pregunta al RAG](#toc0_)

In [20]:
def ask_rag(question, vector_store, k = 3):
    
        """
    Ejecuta un ciclo completo de RAG: recuperación + generación.

    Args:
        question (str): Pregunta del usuario.
        vector_store: Base vectorial configurada previamente.
        k (int): Número de fragmentos a recuperar.

    Returns:
        dict: Resultado con pregunta, contexto, respuesta y fuentes.
    """

        # Paso 1: Recuperar documentos relevantes del vector store.
        retrieved_docs = vector_store.similarity_search(question, k=k)

        # Paso 2: Preparar el contexto concatenando el contenido de los documentos.
        docs_content = "\n\n".join(doc.page_content for doc in retrieved_docs)

        # Paso 3: Construir el mensaje de entrada usando el prompt definido.
        messages = prompt.invoke({
                "question": question,
                "context": docs_content
        })

        # Paso 4: Generar la respuesta con el modelo LLM.
        response = llm.invoke(messages)

        # Construir el resultado como diccionario estructurado.
        return {
                "question": question,
                "context": retrieved_docs,
                "answer": response.content,
                "sources": [doc.metadata.get('source_file', 'unknown') for doc in retrieved_docs]
        }

### <a id='toc3_2_3_'></a>[Mostrar resultados del RAG](#toc0_)

In [21]:
def display_rag_result(result):
    
        """
    Muestra los resultados de una ejecución RAG con formato estructurado.

    Args:
        result (dict): Diccionario con pregunta, contexto, respuesta y fuentes.
    """

        print("\n" + "=" * 60)
        print(f"🤔 PREGUNTA: {result['question']}")
        print("=" * 60)

        print(f"\n📚 CONTEXTO RECUPERADO:")
        print("-" * 30)
        for i, doc in enumerate(result["context"], 1):
                source = doc.metadata.get('source_file', 'unknown')
                preview = doc.page_content[:100].replace('\n', ' ')
                print("\n" + "-" * 30)
                print(f"{i}. {source}: '{preview}...'")

        print("\n" + "-" * 30)
        
        print(f"\n💡 RESPUESTA:")
        print("-" * 30)
        print(result["answer"])

        print(f"\n📄 FUENTES: {', '.join(set(result['sources']))}")

### <a id='toc3_2_4_'></a>[Prueba del RAG](#toc0_)

In [22]:
def test_rag_system(vector_store, test_questions = None):
    
        """
    Ejecuta una batería de pruebas sobre el sistema RAG.

    Args:
        vector_store: Vector store previamente cargado.
        test_questions (list): Preguntas de prueba. Si no se especifican, se usan preguntas por defecto.
    """
      
        if test_questions is None:
                test_questions = [
                        "¿Qué son las APIs REST y cuáles son sus principios fundamentales?",
                        "¿Cómo funciona la autenticación JWT?",
                        "¿Cuáles son los códigos de estado HTTP más comunes?",
                        "¿Qué estrategias de chunking funcionan mejor para documentación técnica?",
                        "¿Cuáles son las mejores prácticas para el manejo de errores en APIs?"
                ]

        print("\n🧪 PRUEBAS DEL SISTEMA RAG")
        print("=" * 50)

        for i, question in enumerate(test_questions, 1):
                print(f"\n🔍 PRUEBA {i}/{len(test_questions)}")
                result = ask_rag(question, vector_store)
                display_rag_result(result)
                print("\n" + "=" * 60)

In [23]:
# Ejemplo individual de uso del sistema RAG
question = "\n¿Qué códigos HTTP se usan para errores del cliente?"
result = ask_rag(question, vector_store)
display_rag_result(result)

# Ejecución del conjunto completo de pruebas
print("\n🚀 INICIANDO PRUEBAS COMPLETAS...")
test_rag_system(vector_store)


🤔 PREGUNTA: 
¿Qué códigos HTTP se usan para errores del cliente?

📚 CONTEXTO RECUPERADO:
------------------------------

------------------------------
1. api_development_standards.md: '## Códigos de Estado HTTP   ### Códigos de Éxito (2xx) - **200 OK**: Solicitud exitosa - **201 Creat...'

------------------------------
2. api_development_standards.md: '### Códigos de Error del Cliente (4xx) - **400 Bad Request**: Solicitud malformada - **401 Unauthori...'

------------------------------
3. api_development_standards.md: '### Métodos HTTP y su Uso   | Método | Propósito | Idempotente | Seguro | |--------|-----------|----...'

------------------------------

💡 RESPUESTA:
------------------------------
Los códigos HTTP usados para errores del cliente son: 400 Bad Request, 401 Unauthorized, 403 Forbidden, 404 Not Found, 409 Conflict y 422 Unprocessable Entity. Estos códigos indican problemas con la solicitud enviada por el cliente.

📄 FUENTES: api_development_standards.md

🚀 INICIANDO PR

## <a id='toc3_3_'></a>[Evaluación del sistema RAG](#toc0_)

### <a id='toc3_3_1_'></a>[ Crea dataset con ground truth basado ÚNICAMENTE en información que realmente existe en los documentos proporcionados](#toc0_)

In [24]:
def create_ground_truth_dataset():
    
    """
    Crea dataset con ground truth basado ÚNICAMENTE en información 
    que realmente existe en los documentos proporcionados.
    """
    
    # Ground truth extraído directamente de los documentos disponibles
    ground_truth_qa = [
        {
            "question": "¿Cuáles son los principios fundamentales de REST?",
            "ground_truth": "Los principios fundamentales de REST son: 1) Cliente-Servidor (separación clara de responsabilidades), 2) Sin Estado (cada solicitud debe contener toda la información necesaria), 3) Cacheable (las respuestas deben ser marcadas como cacheables o no), 4) Interfaz Uniforme (uso consistente de métodos HTTP y URIs), y 5) Sistema en Capas (la arquitectura puede estar compuesta por múltiples capas)."
        },
        {
            "question": "¿Qué códigos HTTP se usan para errores del cliente?",
            "ground_truth": "Los códigos HTTP para errores del cliente (4xx) incluyen: 400 Bad Request (solicitud malformada), 401 Unauthorized (autenticación requerida), 403 Forbidden (acceso denegado), 404 Not Found (recurso no encontrado), 409 Conflict (conflicto con el estado actual del recurso), y 422 Unprocessable Entity (datos de entrada inválidos)."
        },
        {
            "question": "¿Cómo funciona la autenticación JWT?",
            "ground_truth": "JWT (JSON Web Tokens) tiene la estructura Header.Payload.Signature. Los tokens JWT contienen información del usuario codificada y firmada, permitiendo verificación sin consultar la base de datos. Se implementa usando Bearer Token en el header Authorization: Bearer <token>. El header contiene el algoritmo (ej: HS256) y tipo (JWT), mientras el payload incluye datos como sub, name, iat y exp."
        },
        {
            "question": "¿Cuáles son las ventajas y desventajas de microservicios?",
            "ground_truth": "Las ventajas de microservicios incluyen: escalabilidad independiente, tecnologías heterogéneas, y despliegue independiente. Las desventajas son: complejidad de red, gestión de datos distribuidos, y monitoreo complejo."
        },
        {
            "question": "¿Qué métodos HTTP son idempotentes?",
            "ground_truth": "Los métodos HTTP idempotentes son GET (obtener recursos), PUT (actualizar/crear recursos) y DELETE (eliminar recursos). POST y PATCH no son idempotentes."
        },
        {
            "question": "¿Cuáles son las estrategias de rate limiting?",
            "ground_truth": "Las estrategias de rate limiting incluyen: Fixed Window (límite fijo por ventana de tiempo), Sliding Window (ventana deslizante más precisa), Token Bucket (permite ráfagas controladas), y Leaky Bucket (flujo constante de solicitudes). Se implementa con headers como X-RateLimit-Limit, X-RateLimit-Remaining y X-RateLimit-Reset."
        },
        {
            "question": "¿Cuál es el modelo Git Flow obligatorio según las políticas?",
            "ground_truth": "El modelo Git Flow obligatorio incluye: main/master (código de producción), develop (integración de nuevas funcionalidades), feature/* (desarrollo de nuevas características), release/* (preparación de versiones), y hotfix/* (correcciones urgentes de producción). El formato requerido para branches es feature/TICKET-123-descripcion-corta."
        },
        {
            "question": "¿Cuáles son los requisitos mínimos para Pull Requests?",
            "ground_truth": "Los requisitos mínimos obligatorios para Pull Requests son: mínimo 2 revisores aprobados, todos los tests automatizados exitosos, coverage de código mayor al 80%, sin conflictos de merge, y documentación actualizada si aplica."
        }
    ]
    
    return ground_truth_qa

### <a id='toc3_3_2_'></a>[Crea un dataset de evaluación solo con preguntas cuya respuesta puede verificarse con los documentos disponibles (ground truth verificado).](#toc0_)

In [25]:
def create_evaluation_dataset_with_verified_ground_truth(vector_store):
    
        """
    Crea un dataset de evaluación solo con preguntas cuya respuesta puede verificarse 
    con los documentos disponibles (ground truth verificado).
    """
    
        print("\nGenerando dataset con ground truth VERIFICADO...\n")

        # Genera un conjunto de datos que contiene preguntas con sus respuestas correctas esperadas.
        ground_truth_data = create_ground_truth_dataset()

        dataset = []

        # Itera sobre cada ejemplo de ground truth
        for item in ground_truth_data:
                question = item["question"]
                ground_truth = item["ground_truth"]

                # Llama al sistema RAG para obtener una respuesta generada a partir de la base vectorial.
                result = ask_rag(question, vector_store)

                # Extrae el contenido de los documentos usados como contexto en la respuesta.
                contexts = [doc.page_content for doc in result["context"]]

                # Crea una entrada del dataset con pregunta, respuesta generada, contextos y ground truth.
                dataset_entry = {
                        "question": question,
                        "answer": result["answer"],
                        "contexts": contexts,
                        "ground_truth": ground_truth
                }

                dataset.append(dataset_entry)

                # Imprime una confirmación parcial para seguimiento del proceso.
                print(f"✓ Procesada: {question[:50]}...")

        # Muestra la cantidad de ejemplos generados.
        print(f"\nDataset verificado creado con {len(dataset)} ejemplos")

        return dataset

### <a id='toc3_3_3_'></a>[Evalúa el sistema RAG utilizando las métricas definidas por RAGAS](#toc0_)

In [26]:
def evaluate_rag_with_ragas(dataset):
    
        """
    Evalúa el sistema RAG utilizando las métricas definidas por RAGAS.
    Imprime información del proceso y retorna los resultados de evaluación.
    """

        print("\nEVALUACIÓN RAG CON RAGAS")
        print("=" * 40)

        # Configura el modelo de lenguaje para evaluación (sin aleatoriedad) y el modelo de embeddings.
        llm = ChatGoogleGenerativeAI(model = "gemini-1.5-flash", temperature = 0)
        embeddings = GoogleGenerativeAIEmbeddings(model = "models/embedding-001")

        # Lista de métricas RAGAS a utilizar.
        metrics = [faithfulness, answer_relevancy, context_precision, context_recall]

        # Convierte el dataset original en el formato requerido por RAGAS.
        dataset_dict = {
                "question": [item["question"] for item in dataset],
                "answer": [item["answer"] for item in dataset],
                "contexts": [item["contexts"] for item in dataset],
                "ground_truth": [item["ground_truth"] for item in dataset]
        }

        # Crea un dataset compatible con RAGAS a partir del diccionario.
        eval_dataset = Dataset.from_dict(dataset_dict)

        print(f"\nEvaluando {len(dataset)} preguntas...")
        print("Procesando métricas RAGAS...")

        # Ejecuta la evaluación con las métricas seleccionadas, utilizando el modelo LLM y embeddings definidos.
        results = evaluate(
                dataset = eval_dataset,
                metrics = metrics,
                llm = llm,
                embeddings = embeddings
        )

        return results

### <a id='toc3_3_4_'></a>[Resumen de métricas RAGAS promediada](#toc0_)

In [27]:
def display_ragas_summary(results):
    
        """
    Muestra un resumen de métricas RAGAS promediadas.
    Retorna un diccionario con las métricas y el puntaje general.
    """

        print("\nRESULTADOS PROMEDIO RAGAS")
        print("=" * 50)

        try:
                # Si los resultados tienen método to_pandas (formato DataFrame), se calcula el promedio desde ahí.
                if hasattr(results, 'to_pandas'):
                        df = results.to_pandas()
                        
                        # Calcula promedio para cada métrica si está presente en el DataFrame.
                        metrics_data = {
                                'faithfulness': df['faithfulness'].mean() if 'faithfulness' in df.columns else 0,
                                'answer_relevancy': df['answer_relevancy'].mean() if 'answer_relevancy' in df.columns else 0,
                                'context_precision': df['context_precision'].mean() if 'context_precision' in df.columns else 0,
                                'context_recall': df['context_recall'].mean() if 'context_recall' in df.columns else 0
                        }
                else:
                        # Si no es DataFrame, intenta acceder directamente a los atributos de resultado.
                        metrics_data = {
                                'faithfulness': getattr(results, 'faithfulness', 0),
                                'answer_relevancy': getattr(results, 'answer_relevancy', 0),
                                'context_precision': getattr(results, 'context_precision', 0),
                                'context_recall': getattr(results, 'context_recall', 0)
                        }

        except Exception as e:
                # Captura errores durante el acceso a los resultados.
                print(f"Error accediendo resultados: {e}")
                return None

        print("\nMÉTRICAS PROMEDIO:")
        print("-" * 30)

        # Itera sobre las métricas y muestra su valor con una calificación cualitativa.
        for metric_name, score in metrics_data.items():
                # Asigna un estado visual según el valor de la métrica.
                if score > 0.8:
                        status = "🟢 EXCELENTE"
                elif score > 0.7:
                        status = "🟡 BUENO"
                elif score > 0.6:
                        status = "🟠 ACEPTABLE"
                else:
                        status = "🔴 DEFICIENTE"

                # Formatea el nombre de la métrica para mostrarlo con mayúsculas iniciales.
                metric_display = metric_name.replace('_', ' ').title()
                print(f"{metric_display:<20} {score:.3f} {status}")

        # Calcula y muestra el puntaje promedio general.
        overall_score = sum(metrics_data.values()) / len(metrics_data)
        print("-" * 30)
        print(f"{'Score General':<20} {overall_score:.3f}")

        return metrics_data, overall_score

### <a id='toc3_3_5_'></a>[Corre una evalución del RAGAS](#toc0_)

In [28]:
def run_simple_ragas_evaluation(vector_store):
    
        """
    Ejecuta una evaluación simplificada del sistema RAG utilizando RAGAS.
    Muestra un resumen de métricas y retorna los resultados detallados.
    """
    
        print("\nEVALUACIÓN RAGAS")
        print("=" * 45)

        # Genera un dataset a partir de preguntas con ground truth verificable.
        dataset = create_evaluation_dataset_with_verified_ground_truth(vector_store)

        # Evalúa el rendimiento del sistema con las métricas de RAGAS.
        results = evaluate_rag_with_ragas(dataset)

        # Muestra un resumen de los resultados con promedios e interpretación cualitativa.
        metrics_data, overall_score = display_ragas_summary(results)

        # Retorna resultados completos, métricas promedio y score general.
        return results, metrics_data, overall_score

In [29]:
results, metrics_data, overall_score = run_simple_ragas_evaluation(vector_store)


EVALUACIÓN RAGAS

Generando dataset con ground truth VERIFICADO...

✓ Procesada: ¿Cuáles son los principios fundamentales de REST?...
✓ Procesada: ¿Qué códigos HTTP se usan para errores del cliente...
✓ Procesada: ¿Cómo funciona la autenticación JWT?...
✓ Procesada: ¿Cuáles son las ventajas y desventajas de microser...
✓ Procesada: ¿Qué métodos HTTP son idempotentes?...
✓ Procesada: ¿Cuáles son las estrategias de rate limiting?...
✓ Procesada: ¿Cuál es el modelo Git Flow obligatorio según las ...
✓ Procesada: ¿Cuáles son los requisitos mínimos para Pull Reque...

Dataset verificado creado con 8 ejemplos

EVALUACIÓN RAG CON RAGAS

Evaluando 8 preguntas...
Procesando métricas RAGAS...


E0000 00:00:1760853410.871409  203985 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.
E0000 00:00:1760853410.872746  203985 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


Evaluating:   0%|          | 0/32 [00:00<?, ?it/s]

E0000 00:00:1760853411.637579  203985 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.
Retrying langchain_google_genai.chat_models._achat_with_retry.<locals>._achat_with_retry in 2.0 seconds as it raised NotFound: 404 models/gemini-1.5-flash is not found for API version v1beta, or is not supported for generateContent. Call ListModels to see the list of available models and their supported methods..
Retrying langchain_google_genai.chat_models._achat_with_retry.<locals>._achat_with_retry in 2.0 seconds as it raised NotFound: 404 models/gemini-1.5-flash is not found for API version v1beta, or is not supported for generateContent. Call ListModels to see the list of available models and their supported methods..
Retrying langchain_google_genai.chat_models._achat_with_retry.<locals>._achat_with_retry in 2.0 seconds as it raised NotFound: 404 models/gemini-1.5-flash is not found for API version v1beta, or is not supported for generateContent. C


RESULTADOS PROMEDIO RAGAS

MÉTRICAS PROMEDIO:
------------------------------
Faithfulness         nan 🔴 DEFICIENTE
Answer Relevancy     nan 🔴 DEFICIENTE
Context Precision    nan 🔴 DEFICIENTE
Context Recall       nan 🔴 DEFICIENTE
------------------------------
Score General        nan
